<a href="https://colab.research.google.com/github/willywildan/ETLwithPandas/blob/master/ProjectETLwithPandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
#read data
df_participant = pd.read_csv('/dqthon-participants.csv')

#Transform
df_participant['postal_code'] = df_participant['address'].str.extract(r'(\d+)$') #Masukkan regex Anda didalam fungsi extract
df_participant['city'] = df_participant['address'].str.extract(r'(?<=\n)(\w.+)(?=,)') 
df_participant['github_profile'] = 'https://github.com/' + df_participant['first_name'].str.lower() + df_participant['last_name'].str.lower()

#Masukkan regex anda pada parameter pertama dari fungsi replace
#Jika awalan nomor HP berupa angka 62 atau +62 yang merupakan kode telepon Indonesia, maka diterjemahkan ke 0.
#Tidak ada tanda baca seperti kurung buka, kurung tutup, strip⟶ ()-
#Tidak ada spasi pada nomor HP nama kolom untuk menyimpan hasil cleansing pada nomor HP yaitu cleaned_phone_number
df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0')
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '')
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'\s+', '')

In [ ]:
def func(col):
    abbrev_name = "%s%s"%(col['first_name'][0],col['last_name'][0]) #Singkatan dari Nama Depan dan Nama Belakang dengan mengambil huruf pertama
    country = col['country']
    abbrev_institute = '%s'%(''.join(list(map(lambda word: word[0], col['institute'].split())))) #Singkatan dari value di kolom institute
    return "%s-%s-%s"%(abbrev_name,country,abbrev_institute)

df_participant['team_name'] = df_participant.apply(func, axis=1)

def func(col):
	first_name_lower = col['first_name'].lower()
	last_name_lower = col['last_name'].lower()
	institute = ''.join(list(map(lambda word: word[0], col['institute'].lower().split()))) #Singkatan dari nama perusahaan dalam lowercase

	if 'Universitas' in col['institute']:
		if len(col['country'].split()) > 1: #Kondisi untuk mengecek apakah jumlah kata dari country lebih dari 1
					country = ''.join(list(map(lambda word: word[0], col['country'].lower().split())))
		else:
			country = col['country'][:3].lower()
		return "%s%s@%s.ac.%s"%(first_name_lower,last_name_lower,institute,country)

	return "%s%s@%s.com"%(first_name_lower,last_name_lower,institute)

df_participant['email'] = df_participant.apply(func, axis=1)
#Formatting DD-MM-YYYY exam:05 Feb 1991
df_participant['birth_date'] = pd.to_datetime(df_participant['birth_date'], format='%d %b %Y')
#formatting YYYY-MM-DD HH:mm:ss 
df_participant['register_at'] = pd.to_datetime(df_participant['register_time'], unit='s')